In [1]:
# Import libraries
import torch
import time
from environment import PruningEnv
from REINFORCE_agent import REINFORCE_agent
import os
import logging
import numpy as np
logging.basicConfig(level=logging.INFO, 
                    format=('%(levelname)s:' +
                            '[%(filename)s:%(lineno)d]' +
                            ' %(message)s'))

# Define Agent, Training Env, & Hyper-params
env = PruningEnv()
print(env.state_size)
agent = REINFORCE_agent(env.state_size, 512)

M = 20# no reason, number of training episodes
layers_to_prune = [] # will be list of string names
for layer_name, _ in env.model.named_modules():
    if "conv" in layer_name:
        layers_to_prune.append(layer_name)
listacc = np.zeros([4,M])

listnumpruned = np.zeros([4,M])
listreward = np.zeros([4,M])
listflop = np.zeros([4,M])
listvalacc = np.zeros([100])

env._evaluate_model()

Files already downloaded and verified
Files already downloaded and verified
Starting Pre-Training
Training CNN model
Training Done
512


<bound method PruningEnv._evaluate_model of <environment.PruningEnv object at 0x00000189731A1E80>>

In [5]:
print(env._evaluate_model())

Evaluating CNN model
Validation Accuracy: 11.61%
0.1161


In [19]:

for episode in range(M):
    print("=====",episode,"New Episode===========================================================")
    env.load_trained() # reset CNN to full-params
    env._evaluate_model()
    action_reward_buffer = [] # list of (action,reward) tuples per episode

    # single rollout, layer-by-layer CNN scan
    for layer_name in layers_to_prune:
        env.layer_to_process = layer_name
        print("===== Working on", layer_name, "layer =====")
        # get state from orig model (or should we get from pruned model?)
        state = env.get_state()
        
        action = agent.get_action(state)


        action = torch.unsqueeze(action, 0)
        #logging.info("Actions: {}".format(action))
        action_to_index = (action > 0.5).type(torch.int)
        #logging.info("action_to_index sum: {}".format(action_to_index.sum()))
        env.prune_layer(action_to_index)
        print("Calculating reward")
        total_filters = 64*(2**(int(layer_name[-1])-1))
        amount_pruned = action_to_index[0,0:64*(2**(int(layer_name[-1])-1))].sum()
        reward,acc,flop = env._calculate_reward(amount_pruned,total_filters)
        
        #reward = reward * -1
        #it seems this doesnt matter? as long as it heads to 0 then that is the goal maybe this is a lr problem
        action_reward_buffer.append((action, reward))

        
        print("For",layer_name,"pruned",action_to_index[0,0:64*(2**(int(layer_name[-1])-1))].sum())
        listacc[int(layer_name[-1])-1, episode]=acc
        print("FILT",64*(2**(int(layer_name[-1])-1)))
        listnumpruned[int(layer_name[-1])-1, episode] = action_to_index[0,0:64*(2**(int(layer_name[-1])-1))].sum()
        listreward[int(layer_name[-1])-1, episode] = reward
        listflop[int(layer_name[-1])-1, episode] = flop
    # calc cumulative reward, agent learns 
    actions, rewards = zip(*action_reward_buffer) # both var are tuple wrapped
    # actions: tuple->tensor
    actions = torch.squeeze(torch.stack(actions)).type(torch.float)
    agent.update_policy(rewards, actions) 


#PATH = os.getcwd() + '\pruned_50_1_train_after.pth'
#torch.save(env.model.state_dict(), PATH)


===== 0 New Episode===========================================================
Evaluating CNN model
Validation Accuracy: 78.46%
===== Working on conv1 layer =====


INFO:[environment.py:126] Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))


Calculating reward
Training CNN model
Training Done
Evaluating CNN model
Validation Accuracy: 48.05%
Reward: -0.5195000000000001
For conv1 pruned tensor(28)
FILT 64
===== Working on conv2 layer =====


INFO:[environment.py:126] Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))


Calculating reward
Training CNN model
Training Done
Evaluating CNN model
Validation Accuracy: 33.71%
Reward: -0.6629
For conv2 pruned tensor(64)
FILT 128
===== Working on conv3 layer =====


...

INFO:[environment.py:126] Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))


Calculating reward
Training CNN model
Training Done
Evaluating CNN model
Validation Accuracy: 27.60%
Reward: -0.724
For conv3 pruned tensor(143)
FILT 256
===== Working on conv4 layer =====


INFO:[environment.py:126] Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))


Calculating reward
Training CNN model
Training Done
Evaluating CNN model
Validation Accuracy: 27.12%
Reward: -0.7288
For conv4 pruned tensor(261)
FILT 512
Gt = [-2.53066531 -2.08915688 -1.445512   -0.7288    ]


INFO:[REINFORCE_agent.py:82] Actions: tensor([[0.2934, 0.8709, 0.9153,  ..., 0.0833, 0.8855, 0.6579],
        [0.8824, 0.8645, 0.9974,  ..., 0.1404, 0.9876, 0.7973],
        [0.6240, 0.5616, 0.9873,  ..., 0.1661, 0.9700, 0.7777],
        [0.5943, 0.7436, 0.7284,  ..., 0.2254, 0.8143, 0.5285]],
       grad_fn=<SqueezeBackward0>)


In [20]:

print("List of accuraccies",listacc)
print("list of pruned", listnumpruned)
print("list of rewards", listreward)
print("list of flops", listflop)
print("list of valacc", listvalacc)

List of accuraccies [[0.4805 0.4805 0.4805 0.4859 0.4859 0.4859 0.5097 0.5097 0.5097 0.5097
  0.5097 0.5097 0.5097 0.5097 0.5097 0.5097 0.5097 0.5097 0.5097 0.5376]
 [0.3371 0.339  0.339  0.3313 0.3313 0.3279 0.3264 0.3332 0.3332 0.3332
  0.323  0.3275 0.3275 0.3229 0.3229 0.3229 0.3229 0.3229 0.3191 0.3419]
 [0.2343 0.2396 0.243  0.2428 0.2461 0.2425 0.2407 0.2481 0.2481 0.2481
  0.2501 0.2401 0.2439 0.2423 0.2399 0.2424 0.2597 0.2596 0.2575 0.276 ]
 [0.2333 0.2338 0.2397 0.2421 0.2415 0.2477 0.2485 0.2482 0.2488 0.2463
  0.2515 0.2403 0.2442 0.2346 0.2311 0.2377 0.255  0.2507 0.2585 0.2712]]
list of pruned [[ 28.  28.  28.  29.  29.  29.  30.  30.  30.  30.  30.  30.  30.  30.
   30.  30.  30.  30.  30.  31.]
 [ 64.  65.  65.  66.  66.  66.  64.  65.  65.  65.  64.  65.  65.  66.
   66.  66.  66.  66.  65.  65.]
 [135. 135. 135. 135. 136. 136. 137. 136. 136. 135. 137. 134. 138. 139.
  137. 138. 140. 142. 143. 143.]
 [258. 264. 264. 264. 267. 268. 265. 265. 269. 267. 267. 265. 265. 26